# Hosting Yi-6B-Chat on Amazon SageMaker using HuggingFace Text Generation Inference (TGI)

---

This notebook's CI test result for eu-west-1 with Image of "Data Science 3.0" and instance type ml.t3.medium

---


The [Yi series](https://huggingface.co/01-ai) models are the next generation of open-source large language models trained from scratch by 01.AI.For English language capability, the Yi series models ranked 2nd (just behind GPT-4), outperforming other LLMs (such as LLaMA2-chat-70B, Claude 2, and ChatGPT) on the AlpacaEval Leaderboard in Dec 2023.

---
This notebook demonstrates how to deploy Hosting Yi-6B-Chat transformer models using Hugging Face Text Generation Inference (TGI) Deep Learning Container on Amazon SageMaker.

TGI is an open source, high performance inference library that can be used to deploy large language models from Hugging Face’s repository in minutes. 

---




## Setup

### Install the SageMaker Python SDK

First, make sure that the latest version of SageMaker SDK is installed.

In [ ]:
%pip install --upgrade pip --quiet
%pip install "sagemaker>=2.163.0" --quiet

### Setup account and role

Then, we import the SageMaker python SDK and instantiate a `sagemaker_session` which we use to determine the current region and execution role.

In [2]:
import json
import sagemaker
import boto3
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Retrieve the LLM Image URI

We use the helper function `get_huggingface_llm_image_uri()` to generate the appropriate image URI for the Hugging Face Large Language Model (LLM) inference.

The function takes a required parameter `backend` and several optional parameters. The `backend` specifies the type of backend to use for the model, the values can be "lmi" and "huggingface". The "lmi" stands for SageMaker LMI inference backend, and "huggingface" refers to using Hugging Face TGI inference backend.

In [3]:
image_uri = get_huggingface_llm_image_uri(backend="huggingface", region=region)  # or lmi

## Create the Hugging Face Model

Next we configure the `model` object by specifying a unique name, the `image_uri` for the managed TGI container, and the execution role for the endpoint. Additionally, we specify a number of environment variables including the `HF_MODEL_ID` which corresponds to the model from the HuggingFace Hub that will be deployed, and the `HF_TASK` which configures the inference task to be performed by the model.

Additionally, Only if you need authentication, use the 'HF_TOKEN' to provide your Hugging Face token, but only if you need authentication, as per the documentation.
https://huggingface.co/docs/transformers.js/guides/private

In [4]:
import time

model_checkpoint = "01-ai/Yi-6B-Chat"
model_name = f'{model_checkpoint.split("/")[-1]}-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'

# If your model is private, you need to use a Hugging Face authentication token
# Replace 'your_token' with your actual Hugging Face token
# If the model is public, you can omit the 'HF_TOKEN' key-value pair
hub = {
    'HF_MODEL_ID': model_checkpoint,  # Use the variable, not the string 'model_checkpoint'
    'SM_NUM_GPUS': json.dumps(1),
    #'HF_TOKEN': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  # Only if you need authentication
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.1.0"),
	env=hub,
	role=role, )

## Creating a SageMaker Endpoint

Next we deploy the model by invoking the `deploy()` function. Here we use an `ml.g5.16xlarge` instance which come with 1 NVIDIA A10G Tensor Core GPU. 

In [5]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
    instance_type="ml.g5.16xlarge",
	container_startup_health_check_timeout=100,
  )

--------!

## Running Inference

Once the endpoint is up and running, we can evaluate the model using the `predict()` function.

In [20]:
# send request with default min tokens

predictor.predict({
	"inputs": "what is the capital of UK", 
})

[{'generated_text': 'what is the capital of UK?\nLondon is the capital city of the United Kingdom.\n\nWhat is the capital of England'}]

In [21]:
# Assuming predictor is already set up with your model
# Setting up a large output tokens of 200
input_prompt = {
    "inputs": "write a story about a king and slaves in a village",
    "parameters": {"do_sample": True, "max_new_tokens": 200, "temperature": 0.7, "watermark": True},
    
}

response = predictor.predict(input_prompt)

# Process the response as needed
print(response)


[{'generated_text': "write a story about a king and slaves in a village.\nOnce upon a time, there was a kingdom ruled by a wise and just king. The king had a deep compassion for his people and treated them with kindness and respect. He believed that everyone, regardless of their station in life, deserved to live a life of dignity and freedom.\n\nIn the king's kingdom, there was a small village where the king's rule extended. The villagers were hardworking and prosperous, and they all worked together to build a harmonious community. The villagers had various professions, from farmers to craftsmen, and they all lived in peace and contentment.\n\nOne day, a group of traders from a neighboring kingdom arrived in the village. These traders were known for their deceitful ways and for their practice of enslaving the people they encountered. They spotted some of the village craftsmen, who were skilled in various trades, and offered them a tempting deal.\n\nThe traders offered to buy the crafts

## Cleaning Up

After you've finished using the endpoint, it's important to delete it to avoid incurring unnecessary costs.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

## Conclusion

In this tutorial, we used a TGI container to deploy Yi-6B-chat using 1 GPUs on a SageMaker `ml.g5.16xlarge` instance. With Hugging Face's Text Generation Inference and SageMaker Hosting, you can easily host large language models like GPT-NeoX, flan-t5-xxl, and LLaMa.